In [40]:
import tensorflow.compat.v1 as tf
import pandas as pd
tf.disable_eager_execution()

In [41]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('banknote.csv')
df

In [ ]:
print(df.isnull().sum())

In [44]:
one_hot=pd.get_dummies(df['class'])
df.drop(['class'], axis=1, inplace=True)
df=pd.concat([df, one_hot], axis=1)

In [45]:
X=df.drop([0, 1], axis=1)
Y=df[[0, 1]]
X, Y=shuffle(X, Y, random_state=0)
x_train, x_test, y_train, y_test=train_test_split(X, Y, test_size=0.33, random_state=0)

In [ ]:
print(X.shape[1])

In [47]:
x=tf.placeholder(tf.float32, [None, 4])
y=tf.placeholder(tf.float32, [None, 2])

In [48]:
w={'h1':tf.Variable(tf.truncated_normal([4, 4])),
    'h2':tf.Variable(tf.truncated_normal([4, 4])),
    'h3':tf.Variable(tf.truncated_normal([4, 4])),
    'h4':tf.Variable(tf.truncated_normal([4, 4])),
    'out':tf.Variable(tf.truncated_normal([4, 2]))}

In [49]:
b={'b1':tf.Variable(tf.truncated_normal([4])),
    'b2':tf.Variable(tf.truncated_normal([4])),
    'b3':tf.Variable(tf.truncated_normal([4])),
    'b4':tf.Variable(tf.truncated_normal([4])),
    'out':tf.Variable(tf.truncated_normal([2]))}

In [50]:
layer1=tf.nn.sigmoid(tf.add(tf.matmul(x, w['h1']), b['b1']))
layer2=tf.nn.sigmoid(tf.add(tf.matmul(layer1, w['h2']), b['b2']))
layer3=tf.nn.sigmoid(tf.add(tf.matmul(layer2, w['h3']), b['b3']))
layer4=tf.nn.relu(tf.add(tf.matmul(layer3, w['h4']), b['b4']))
y_predict=tf.add(tf.matmul(layer4, w['out']), b['out'])

In [51]:
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_predict, labels=y))
optimizer=tf.train.GradientDescentOptimizer(0.3)
train=optimizer.minimize(loss)
delta=tf.equal(tf.argmax(y, 1), tf.argmax(y_predict, 1))
accuracy=tf.reduce_mean(tf.cast(delta, tf.float32))

In [ ]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(1, 1001, 1) :
  sess.run(train, feed_dict={x:x_train, y:y_train})
  loss_=sess.run(loss, feed_dict={x:x_train, y:y_train})
  accuracy_=sess.run(accuracy, feed_dict={x:x_train, y:y_train})
  print(f'epoch={epoch} -> loss={loss_} -> accuracy={accuracy_}')

In [ ]:
loss_=sess.run(loss, feed_dict={x:x_test, y:y_test})
accuracy_=sess.run(accuracy, feed_dict={x:x_test, y:y_test})
print(loss_, accuracy_)
sess.close()